In [1]:
import pandas as pd
import numpy as np
file_to_load = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


# Player Count
* Display the total number of players

In [2]:
player_count = purchase_data["SN"].nunique()
player_num_df = pd.DataFrame({"Total Players" : [player_count]})
player_num_df 

,Total Players
0,576


# Purchasing Analysis (Total)
* Run Basic Calculations to obtain number of unique items, averge price, etc.
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [3]:
uni_item = purchase_data["Item Name"].nunique()
average_purchase_price = purchase_data["Price"].mean()
total_num_purchase = purchase_data["Item ID"].count()
total_reveneu = purchase_data["Price"].sum()
purchasing_analysis_df = pd.DataFrame({"Number of Unique Items" : [uni_item],
                                        "Average Price" : [average_purchase_price],
                                        "Number of Purchases": [total_num_purchase],
                                        "Total Revenue" : [total_reveneu]})                                       

purchasing_analysis_df["Average Price"]=purchasing_analysis_df["Average Price"].map("${:.2f}".format)
purchasing_analysis_df["Total Revenue"]=purchasing_analysis_df["Total Revenue"].map("${:,.2f}".format)
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


#  Gender Demographics
* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [4]:
gender_df = purchase_data[["SN","Gender"]]
gender_df=gender_df.drop_duplicates(subset="SN")
gender_df=gender_df.reset_index()

gender_dem_df=pd.DataFrame(gender_df["Gender"].value_counts())
gender_dem_df["Percentage of Players"] = gender_dem_df["Gender"]*100  /player_count
gender_dem_df = gender_dem_df.rename(columns={"Gender" : "Total Count"})
gender_dem_df = gender_dem_df [["Percentage of Players", "Total Count"]]
gender_dem_df["Percentage of Players"] = gender_dem_df["Percentage of Players"].map("{:.2f}%".format)

gender_dem_df

,Percentage of Players,Total Count
Male,84.03%,484
Female,14.06%,81
Other / Non-Disclosed,1.91%,11


#  Purchasing Analysis (Gender)
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [5]:
gender_gp = purchase_data.groupby("Gender")
gender_analysis_df = pd.DataFrame({"Purchase Count" : gender_gp["Purchase ID"].count(),
                                  "Average Purchase Price": gender_gp["Price"].mean(),
                                  "Total Purchase Value": gender_gp["Price"].sum(),
                                  "Avg Purchase Total Per Person": gender_gp["Price"].sum()/gender_dem_df["Total Count"] })
gender_analysis_df["Average Purchase Price"] =gender_analysis_df["Average Purchase Price"].map("${:.2f}".format)
gender_analysis_df["Total Purchase Value"] =gender_analysis_df["Total Purchase Value"].map("${:,.2f}".format)
gender_analysis_df["Avg Purchase Total Per Person"] =gender_analysis_df["Avg Purchase Total Per Person"].map("${:.2f}".format)
gender_analysis_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


# Age Demographics
* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Optional: round the percentage column to two decimal points
* Display Age Demographics Table

In [6]:
age_df = purchase_data[["SN","Age","Gender"]]
age_df=age_df.drop_duplicates(subset="SN")
age_df=age_df.reset_index()

age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_df["Age Group"] = pd.cut(age_df["Age"],age_bins,labels = group_names)
age_gp = age_df.groupby("Age Group")
age_demographics_df = pd.DataFrame({"Total Count": age_gp["SN"].count(),
                            "Percentage of Players":  age_gp["SN"].count()* 100 /player_count})
age_demographics_df["Percentage of Players"] = age_demographics_df["Percentage of Players"].map("{:.2f}%".format)

age_demographics_df
                                    


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


# Purchasing Analysis (Age)
* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [7]:
SN_spend = purchase_data.groupby("SN")          #get total spend for each player, to calculate purchase per person
SN_spend_df = pd.DataFrame({"Total Spend":SN_spend["Price"].sum()})
SN_spend_df=SN_spend_df.reset_index()
age_spend = pd.merge(age_df,SN_spend_df)
age_spend["Age Group"] = pd.cut(age_df["Age"],age_bins,labels = group_names)
age_spend_gp = age_spend.groupby("Age Group")
age_spend_gp["Total Spend"].sum()

purchase_data["Age Group"] = pd.cut(purchase_data["Age"],age_bins,labels = group_names)  #used to calculate average purchase price
purchase_data["Age Group"].count()                                                      #purchase count and total purchase value     
purchase_gp = purchase_data.groupby("Age Group")          

age_analysis_df = pd.DataFrame({"Purchase Count": purchase_gp["Purchase ID"].count(),                            
                                "Average Purchase Price": purchase_gp["Price"].mean(),
                                "Total Purchase Value": purchase_gp["Price"].sum(),
                                "Avg Purchase Total Per Person": age_spend_gp["Total Spend"].sum()/age_spend_gp["SN"].count() })

age_analysis_df["Average Purchase Price"] =age_analysis_df["Average Purchase Price"].map("${:.2f}".format)
age_analysis_df["Total Purchase Value"] =age_analysis_df["Total Purchase Value"].map("${:,.2f}".format)
age_analysis_df["Avg Purchase Total Per Person"] =age_analysis_df["Avg Purchase Total Per Person"].map("${:.2f}".format)


age_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Purchase Total Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


#  Top Spenders
* Run basic calculations to obtain the results in the table below
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the summary data frame

In [8]:
SN_gp = purchase_data.groupby("SN")
spenders_df = pd.DataFrame({"Purchase Count" : SN_gp["Price"].count(),
                            "Average Purchase Price" : SN_gp["Price"].mean(),
                            "Total Purchase Value" : SN_gp["Price"].sum()})
top_spenders_df = spenders_df.sort_values("Total Purchase Value", ascending = False)
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].map("${:.2f}".format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].map("${:.2f}".format)
top_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


# Most Popular Items
* Retrieve the Item ID, Item Name, and Item Price columns
* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value
* Create a summary data frame to hold the results
* Sort the purchase count column in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the summary data frame

In [9]:
item_gp = purchase_data.groupby(["Item ID", "Item Name"])
item_analysis = pd.DataFrame({"Purchase Count": item_gp["Price"].count(),
                              "Item Price": item_gp["Price"].mean(),
                              "Total Purchase Value": item_gp["Price"].sum()})
top_popular_item = item_analysis.sort_values("Purchase Count",ascending = False )
top_popular_item["Item Price"] = top_popular_item["Item Price"].map("${:.2f}".format)
top_popular_item["Total Purchase Value"] = top_popular_item["Total Purchase Value"].map("${:.2f}".format)
top_popular_item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


# Most Profitable Items
* Sort the above table by total purchase value in descending order
* Optional: give the displayed data cleaner formatting
* Display a preview of the data frame

In [10]:
top_profit_item = item_analysis.sort_values("Total Purchase Value",ascending = False )
top_profit_item["Item Price"] = top_profit_item["Item Price"].map("${:.2f}".format)
top_profit_item["Total Purchase Value"] = top_profit_item["Total Purchase Value"].map("${:.2f}".format)

top_profit_item.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


# Three observable trends
* Majority of the players who make at least one purchasing are male (83.59%)
* Majority of the players who make at least one purchasing are within age range 20 to 24.
* Although player aged 35-39 only take 5.38% of the players who make at least one purchasing, they spent the most amount in average per person.